In [15]:
from sentence_transformers import SentenceTransformer
import pickle
import pandas as pd
import sys
from os import path
import os

# setting path
sys.path.append('../app')

from src.scripts.similarity import get_similar_works

models_path = "../app/models"
embeddings_path = "../app/embeddings"
qp = "queries/"

# Récupération du modèle non fine-tuné
model_1 = SentenceTransformer(f"{models_path}/sentence_similarity_model")

model_2 = SentenceTransformer(f"{models_path}/sentence_similarity_model_FT")

dataset = pd.read_csv("../Data/movie_synopsis.csv")

with open(f"{embeddings_path}/embeddings_corpus_movie", "rb") as f:
    embeddings = pickle.load(f)

In [16]:
def read(typ):
    return pd.read_csv(f"{qp}{typ}.tsv", sep="\t")

In [17]:
desc = read("desc")

syns = read("syns")

query = read("query")

dfs = {'descriptions':desc,'synopsis':syns,'queries':query}

In [26]:
def eval(typ,df, model, mpath):
    score = 0
    if not path.exists(f"results/{mpath}"):
        os.mkdir(f"results/{mpath}")
    for index, row in df.iterrows():
        title = row['title']
        syn = row['synopsis']
        suggestions = [res['title'] for res in get_similar_works(user_input = syn, oeuvres = embeddings, model = model)]
        if not path.exists(f"results/{mpath}/{typ}"):
            os.mkdir(f"results/{mpath}/{typ}")
        with open(f"results/{mpath}/{typ}/{title}.txt", "w") as f:
            for res in suggestions:
                print(res, file=f)
        if title in suggestions:
            score+=1
    score = score/len(df)

    return score


In [27]:
for k,df in dfs.items():
    print(f"Score obtenu sur les synopsis de type {k} : {eval(k,df, model_1, 'nonFT')}")

['Date Movie', 'Faintheart', 'The Quiet American', "Captain Corelli's Mandolin", 'I Could Never Be Your Woman']
['WordWorld: Rocket to the Moon', 'Little Monsters', 'The Adventures of Pinocchio', 'Nichts geht mehr', 'Baby Einstein: Neighborhood Animals']
['10,000 BC', 'Big City', 'Antarctica', 'The Celestine Prophecy', 'Man in the Chair']
['I Could Never Be Your Woman', 'Inkheart', 'Red Eye', 'The Man Who Knew Too Much', 'Date Movie']
['Two Brothers', 'Alpha and Omega', 'Bambi', 'Legends of the Fall', 'Unstable Fables: Tortoise vs. Hare']
['Animal Farm', 'Disney Princess Enchanted Tales: Follow Your Dreams', 'Bolero', "A Midsummer Night's Dream", 'Shrek 2']
['Dragonslayer', 'The Truth About Demons', 'The Nutty Professor', 'House of Frankenstein', 'The Last Lear']
['Bulletproof Monk', 'The Big Brawl', 'The Karate Kid', 'The Last Dragon', 'The Foot Fist Way']
['Animal Farm', "Pooh's Grand Adventure: The Search for Christopher Robin", 'I Could Never Be Your Woman', 'Marley & Me', '10,000 

In [28]:
for k,df in dfs.items():
    print(f"Score obtenu sur les synopsis de type {k} : {eval(k,df, model_2, 'FT')}")

['Date Movie', 'I Could Never Be Your Woman', 'The Quiet American', 'Faintheart', "Captain Corelli's Mandolin"]
['Nichts geht mehr', 'Little Monsters', 'WordWorld: Rocket to the Moon', 'The Brave Little Toaster', 'Baby Einstein: Neighborhood Animals']
['10,000 BC', 'Big City', 'The Celestine Prophecy', 'Antarctica', 'Legends of the Fall']
['I Could Never Be Your Woman', 'Inkheart', 'The Man Who Knew Too Much', 'Date Movie', 'Red Eye']
['Two Brothers', 'Alpha and Omega', 'Bambi', 'Legends of the Fall', 'The Leopard Son']
["A Midsummer Night's Dream", 'Animal Farm', 'Faintheart', 'Bolero', 'Shrek 2']
['The Nutty Professor', 'Dragonslayer', 'The Truth About Demons', 'House of Frankenstein', 'The Last Lear']
['Bulletproof Monk', 'The Big Brawl', 'The Karate Kid', 'The Last Dragon', 'The Foot Fist Way']
['Animal Farm', 'I Could Never Be Your Woman', "Pooh's Grand Adventure: The Search for Christopher Robin", 'Marley & Me', 'All Roads Lead Home']
['Slither', 'Within the Woods', 'The Signal',